In [40]:
from selenium import webdriver
import pandas as pd
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service


In [53]:
def listToString(list1):
    str1 = " "
    return str1.join(list1)

chrome_driver_path = r"C:\Users\YSK\Documents\Drivers\chromedriver.exe"
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service)
driver.maximize_window()

j= 2
house_list = []
while j<=22:

    driver.get(f"https://www.emlakjet.com/satilik-konut/kayseri-kocasinan/{j}/") 
    i= 1

    wait = WebDriverWait(driver, 30)
    while i<=27:
        if i in (3, 8, 9, 11, 14):
            i=i+1
            continue
        details_dict = {}

        try:
            xpath = f'//*[@id="listing-search-wrapper"]/div[{str(i)}]'
            house_card = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
            actions = ActionChains(driver)
            actions.move_to_element(house_card).click().perform()
        except:
            i=i+1
            continue

        try:
            house_details = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, '_35T4WV')))
            actions = ActionChains(driver)
            actions.move_to_element(house_details[0]).click().perform()
            details = []
            for detail in house_details:
                details.append(detail.text)

            for item in details:
                if '\n' in item:
                    key, value = item.split('\n', 1)
                    details_dict[key] = value

            house_price = driver.find_element(By.CLASS_NAME, 'R-RKDB')
            details_dict['House Price'] = house_price.text

            house_location = driver.find_element(By.XPATH, '//div[@class="_3VQ1JB"]/p')
            details_dict['House Location'] = house_location.text

        except:
            i+=1
            driver.execute_script("window.history.go(-1)")
            time.sleep(3)
            continue

        house_list.append(details_dict)
        print(i,j)
        i = i+1
        driver.execute_script("window.history.go(-1)")
        time.sleep(3)
    j+= 1

driver.quit()

1 2
2 2
4 2
5 2
6 2
7 2
10 2
12 2
13 2
16 2
17 2
18 2
19 2
20 2
21 2
22 2
23 2
24 2
25 2
26 2
27 2
28 2
29 2
1 3
2 3
4 3
5 3
6 3
7 3
10 3
12 3
13 3
15 3
16 3
17 3
18 3
19 3
20 3
21 3
22 3
23 3
24 3
25 3
26 3
27 3
28 3
29 3
1 4
2 4
4 4
5 4
6 4
7 4
10 4
12 4
13 4
15 4
16 4
17 4
18 4
19 4
20 4
21 4
22 4
23 4
24 4
25 4
26 4
27 4
28 4
29 4
1 5
2 5
4 5
5 5
6 5
7 5
12 5
13 5
15 5
16 5
17 5
18 5
19 5
20 5
21 5
22 5
23 5
24 5
25 5
26 5
27 5
28 5
29 5
1 6
2 6
4 6
5 6
6 6
7 6
12 6
13 6
15 6
16 6
17 6
18 6
19 6
20 6
21 6
22 6
23 6
24 6
25 6
26 6
27 6
28 6
29 6
1 7
2 7
4 7
5 7
6 7
7 7
10 7
12 7
13 7
16 7
17 7
18 7
19 7
20 7
21 7
22 7
23 7
24 7
25 7
26 7
27 7
28 7
29 7
1 8
2 8
4 8
5 8
6 8
7 8
10 8
12 8
13 8
16 8
17 8
18 8
19 8
20 8
21 8
22 8
23 8
24 8
25 8
26 8
27 8
28 8
29 8
1 9
2 9
4 9
5 9
6 9
7 9
10 9
12 9
13 9
16 9
17 9
18 9
19 9
20 9
21 9
22 9
23 9
24 9
25 9
26 9
27 9
28 9
29 9
1 10
2 10
4 10
5 10
6 10
7 10
12 10
13 10
15 10
16 10
17 10
18 10
19 10
20 10
21 10
22 10
23 10
24 10
25 10
26 10
27 1

In [54]:
df = pd.DataFrame(house_list)
df.to_csv('kocasinan_houses_data.csv')
print("CSV file created.")

CSV file created.
